___
<img style="float: right; margin: 0px 0px 15px 15px;" src="https://www.carrerasenlinea.mx/logos/original/logo-instituto-tecnologico-y-de-estudios-superiores-de-occidente.png" width="150px" height="100px" />



# <font color= paletvioletred > Laboratorio 4 - Analisis Fundamental</font>




### Integrantes: 
- <Strong> Chavez Jacobo Diana Teresa </Strong> 


- <Strong> Calderón Erape Jazmín </Strong>

#### 14 Abril 2019

___
### <Strong> <font color = skyblue> I. Motivación del Laboratorio </Strong>

___
### <Strong> <font color = skyblue> II. Objetivos específicos </Strong>

In [1]:
# -- ------------------ Importar modulos a utilizar ------------------ -- #

import pandas as pd
from oandapyV20 import API
import oandapyV20.endpoints.instruments as instruments
#import plotly.graph_objs as go
#import plotly as py
from datetime import datetime, date, time, timedelta
import calendar

In [2]:
# -- ------------------ Parametros para Data Frame ------------------ -- #

pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', 200)
pd.set_option('display.width', 1000)

In [3]:
# -- ------------------ Parametros para OANDA ------------------ -- #

A1_OA_Da = 16                     # Day Align
A1_OA_Ta = "America/Mexico_City"  # Time Align

A1_OA_Ai = "101-004-2221697-001"  # Id de cuenta
A1_OA_At = "practice"             # Tipo de cuenta

A1_OA_In = "USD_MXN"              # Instrumento
A1_OA_Gn = "M5"                   # Granularidad de velas

A1_OA_Ak =  "6ff" + "6703d22fd972709992df543492afe" + "-f954a9914f9b39f72fba177a23eb243" + "b" #mi api


In [4]:
# -- ------------------ Estructuras de datos ------------------ -- #



# -- Paso 1 -- Data Frame (df1_precios) descargando todos los precios, segun fechas y granularidad
# -- -- Fecha, Apertura, Maximo, Minimo, Cierre.

# -- Paso 2 -- en Data Frame (df2_operaciones)
# -- -- Fecha, Folio (1 a n), Operacion (1 = compra, -1 = venta), Unidades (0 a n),
# -- -- Margen (0 a n), Comentario ("razon con base al indicador")
# -- -- Precio_apertura, Precio_cierre

# -- Paso 3 -- en Data Frame Cuenta (df3_cuenta)
# -- -- Fecha, Capital (Efectivo $), Flotante ($),
# -- -- Balance (Capital+Flotante), Rend_Balance_acm, Comentario ("Se abrio operacion: compra/venta",
# -- -- "Se cerro operacion: Con perdia de: / Con ganancia de: ", 

# -- Paso 4 -- Generar un vector de fechas, de F1 hasta F2

par0_cap = 100000   # Capital a utilizar $100,000 Usd
par1_mar = 1000000  # Unidades por operacion maximo 1'000,000


# -- Criterios para trading
# -- para cerrar una operacion: 1) TP o SL, 2) Regla con indicador

indice = [1, 2, 3, 4, 5]
columnas = ['A','B', 'C', 'D']
df = pd.DataFrame(index=indice, columns=columnas)
df = df.fillna(0) # -- llenar todas las celdas con 0s

In [5]:
# -- ------------------ Inicializar API de OANDA ------------------ -- #

api = API(access_token=A1_OA_Ak)

In [6]:
# -- ------------------ Función de fechas para descargar precios ------------------ -- #

from datetime import date, datetime, timedelta
from dateutil.relativedelta import relativedelta

def date_range(start_date, end_date, increment, period):
    result = []
    nxt = start_date
    delta = relativedelta(**{period:increment})
    while nxt <= end_date:
        result.append(nxt)
        nxt += delta
    return result

In [7]:
# -- ------------------ vector de fechas por periodos de 15 dias ------------------ -- #

start_date = date(2017, 4, 14)
end_date = date(2019, 4, 10)
fechas = date_range(start_date, end_date, 15, 'days')

F1 = fechas[0].strftime('%Y-%m-%dT%H:%M:%S')
F2 = fechas[1].strftime('%Y-%m-%dT%H:%M:%S')

___
### <Strong> <font color = skyblue> DataFrame1 Precios peso-dólar </Strong>

In [8]:
params = {"granularity": A1_OA_Gn, "price": "M", "dailyAlignment": A1_OA_Da,
          "alignmentTimezone": A1_OA_Ta, "from": F1, "to": F2}

A1_Req1 = instruments.InstrumentsCandles(instrument=A1_OA_In, params=params)
A1_Hist = api.request(A1_Req1)

In [9]:
lis = []
for i in range(len(A1_Hist['candles'])-1):
        lis.append({'TimeStamp': A1_Hist['candles'][i]['time'],
                      'Open': A1_Hist['candles'][i]['mid']['o'],
                      'High': A1_Hist['candles'][i]['mid']['h'],
                      'Low': A1_Hist['candles'][i]['mid']['l'],
                      'Close': A1_Hist['candles'][i]['mid']['c']})

pd_hist = pd.DataFrame(lis)
pd_hist = pd_hist[['TimeStamp', 'Open', 'High', 'Low', 'Close']]
pd_hist['TimeStamp'] = pd.to_datetime(pd_hist['TimeStamp'])

In [10]:
# Muestra de precios históricos del peso dólar.

pd_hist

,TimeStamp,Open,High,Low,Close
0,2017-04-14 04:00:00,18.57634,18.57638,18.57450,18.57556
1,2017-04-14 04:05:00,18.57564,18.57619,18.57479,18.57558
2,2017-04-14 04:10:00,18.57560,18.57568,18.57540,18.57547
3,2017-04-14 04:15:00,18.57550,18.57550,18.57534,18.57538
4,2017-04-14 04:20:00,18.57536,18.57560,18.56932,18.57492
...,...,...,...,...,...
3070,2017-04-28 20:30:00,18.82808,18.82991,18.82721,18.82988
3071,2017-04-28 20:35:00,18.82985,18.83002,18.82970,18.82970
3072,2017-04-28 20:40:00,18.82972,18.82973,18.82614,18.82672
3073,2017-04-28 20:45:00,18.82666,18.82719,18.82492,18.82542


___
### <Strong> <font color = skyblue> Indicador: Consumer Price Index ex Food & Energy (MoM)</Strong>

In [49]:
indicador = pd.read_csv('https://raw.githubusercontent.com/Dianachav/bebe/master/Consumer%20Price%20Index%20ex%20Food%20%26%20Energy%20(MoM).csv')
pd.set_option('display.max_rows', 25)
indicador

,DateTime,Actual,Consensus,Previous
0,4/10/19 12:30,0.1,0.2,0.1
1,3/12/19 12:30,0.1,0.2,0.2
2,2/13/19 13:30,0.2,0.2,0.2
3,1/11/19 13:30,0.2,0.2,0.2
4,12/12/18 13:30,0.2,0.2,0.2
5,11/14/18 13:30,0.2,0.2,0.1
6,10/11/18 12:30,0.1,0.2,0.1
7,9/13/18 12:30,0.1,0.2,0.2
8,8/10/18 12:30,0.2,0.2,0.2
9,7/12/18 12:30,0.2,0.2,0.2


___
### <Strong> <font color = skyblue> Escenarios </Strong>

In [12]:
escenarios = []

for i in range(len(indicador['Actual'])):
    if indicador['Actual'][i]>= indicador['Consensus'][i]>=indicador['Previous'][i]: #Escenario A
        escenarios.append('A')
    elif indicador['Actual'][i]>= indicador['Consensus'][i]<=indicador['Previous'][i]:#Escenario B
        escenarios.append('B')
    elif indicador['Actual'][i]<= indicador['Consensus'][i]>=indicador['Previous'][i]: #Escenario C
        escenarios.append('C')
    elif indicador['Actual'][i]<= indicador['Consensus'][i]<=indicador['Previous'][i]: #Escenario D
        escenarios.append('D')

In [13]:
tabla=pd.DataFrame(columns=['DateTime','Actual','Consensus', 'Previous', 'Scene'])
tabla['DateTime'] = indicador['DateTime']
tabla['Actual'] = indicador['Actual']
tabla['Consensus'] = indicador['Consensus']
tabla['Previous'] = indicador['Previous']
tabla['Scene'] = escenarios
pd.set_option('display.max_rows', 25)
tabla.set_index('DateTime')


,Actual,Consensus,Previous,Scene
DateTime,,,,
4/10/19 12:30,0.1,0.2,0.1,C
3/12/19 12:30,0.1,0.2,0.2,C
2/13/19 13:30,0.2,0.2,0.2,A
1/11/19 13:30,0.2,0.2,0.2,A
12/12/18 13:30,0.2,0.2,0.2,A
11/14/18 13:30,0.2,0.2,0.1,A
10/11/18 12:30,0.1,0.2,0.1,C
9/13/18 12:30,0.1,0.2,0.2,C
8/10/18 12:30,0.2,0.2,0.2,A


___
### <Strong> <font color = skyblue> Ventanas </Strong>

In [50]:
def ventana(F1,F2):
    params = {"granularity": A1_OA_Gn, "price": "M", "dailyAlignment": A1_OA_Da,
              "alignmentTimezone": A1_OA_Ta, "from": F1, "to": F2}

    A1_Req1 = instruments.InstrumentsCandles(instrument=A1_OA_In, params=params)
    A1_Hist = api.request(A1_Req1)
    lista = []

    for i in range(len(A1_Hist['candles'])-1):
            lista.append({'TimeStamp': A1_Hist['candles'][i]['time'],
                          'Open': A1_Hist['candles'][i]['mid']['o'],
                          'High': A1_Hist['candles'][i]['mid']['h'],
                          'Low': A1_Hist['candles'][i]['mid']['l'],
                          'Close': A1_Hist['candles'][i]['mid']['c']})

    pd_hist = pd.DataFrame(lista)
    pd_hist = pd_hist[['TimeStamp', 'Open', 'High', 'Low', 'Close']]
    pd_hist['TimeStamp'] = pd.to_datetime(pd_hist['TimeStamp'])
    return(pd_hist)

___
### <Strong> <font color = skyblue> Características </Strong>

$C1$ = $High_{max}$ - $Low_{min}$  -> Volatilidad

$C2$ = $Open_{t0}$ - $Close_{t10}$  -> Dirección

$C3$ = $Open_{t0}$ - $Low_{t10}$   -> Draw Down

$C4$ = $Open_{t0}$ - $High_{t10}$   -> Draw Up

Donde:

${max}$ = Máximo valor de la ventana de precios

${min}$ = Mínimo valor de la ventana de precios

$t_{0}$ = Tiempo de ocurrencia del indicador

$t_{10}$ = Último dato de la ventana de precios

In [51]:
# -- --------------------------------------Ventana indicador 1 ---------------------------------- #
# -- ------------------------------------------------------------- --------------------------- -- #
v = ventana(F1 = "2019-04-10T11:40:00Z",F2 = "2019-04-10T13:30:00Z")
v

,TimeStamp,Open,High,Low,Close
0,2019-04-10 11:40:00,18.85804,18.85811,18.85555,18.85556
1,2019-04-10 11:45:00,18.85558,18.85976,18.85188,18.85188
2,2019-04-10 11:50:00,18.85190,18.86140,18.85189,18.85867
3,2019-04-10 11:55:00,18.85873,18.86792,18.85686,18.86434
4,2019-04-10 12:00:00,18.86434,18.86434,18.85757,18.86193
5,2019-04-10 12:05:00,18.86194,18.86514,18.86090,18.86200
6,2019-04-10 12:10:00,18.86202,18.86786,18.86156,18.86509
7,2019-04-10 12:15:00,18.86506,18.87281,18.86497,18.86878
8,2019-04-10 12:20:00,18.86879,18.86945,18.86004,18.86416
9,2019-04-10 12:25:00,18.86420,18.86830,18.86191,18.86456


In [52]:
def caract(v):
    #v = ventana(F1 = "2019-04-10T11:40:00Z",F2 = "2019-04-10T13:30:00Z")
    o = v['Open'].astype(float)
    c = v['Close'].astype(float)
    l = v['Low'].astype(float)
    h = v['High'].astype(float)

    caract1= max(h)-min(l)
    caract2= o[10]-c[20]
    caract3= o[10]-l[20]
    caract4= o[10]-h[20]
    c1=round(caract1,3)
    c2=round(caract2,3)
    c3=round(caract3,3)
    c4=round(caract4,3)
    return(c1,c2,c3,c3)

In [53]:
caract(v)


(0.06, 0.021999999999999999, 0.025000000000000001, 0.025000000000000001)

In [54]:
# -- --------------------------------------Ventana indicador 2 ---------------------------------- #
# -- ------------------------------------------------------------- --------------------------- -- #

v = ventana(F1 = "2019-03-12T11:40:00Z",F2 = "2019-03-12T13:30:00Z")


In [55]:
caract(v)

(0.141, 0.063, 0.073999999999999996, 0.073999999999999996)

In [56]:
# -- --------------------------------------Ventana indicador 3 ---------------------------------- #

v = ventana(F1 = "2019-02-13T12:40:00Z",F2 = "2019-02-13T14:30:00Z")

In [57]:
caract(v)

(0.102, -0.045999999999999999, -0.043999999999999997, -0.043999999999999997)

In [58]:
# -- --------------------------------------Ventana indicador 4 ---------------------------------- #

v = ventana(F1 = "2019-01-11T12:40:00Z",F2 = "2019-01-11T14:30:00Z")


In [59]:
caract(v)

(0.089, -0.036999999999999998, -0.017999999999999999, -0.017999999999999999)

In [60]:
# -- --------------------------------------Ventana indicador 5 ---------------------------------- #

v = ventana(F1 = "2018-12-12T12:40:00Z",F2 = "2018-12-12T14:30:00Z")


In [61]:
caract(v)

(0.119, 0.035000000000000003, 0.051999999999999998, 0.051999999999999998)

In [62]:
# -- --------------------------------------Ventana indicador 6 ---------------------------------- #

v = ventana(F1 = "2018-11-14T12:40:00Z",F2 = "2018-11-14T14:30:00Z")


In [63]:
caract(v)

(0.128, 0.036999999999999998, 0.053999999999999999, 0.053999999999999999)

In [70]:
# -- --------------------------------------Ventana indicador 7 ---------------------------------- #

v = ventana(F1 = "2018-10-11T11:40:00Z",F2 = "2018-10-11T13:30:00Z")


In [71]:
caract(v)

(0.141, 0.040000000000000001, 0.042000000000000003, 0.042000000000000003)

In [22]:
# -- --------------------------------------Ventana indicador 8 ---------------------------------- #

v8 = ventana(F1 = "2018-09-13T11:40:00Z",F2 = "2018-09-13T13:30:00Z")
v8_o = v8['Open'].astype(float)
v8_c = v8['Close'].astype(float)
v8_l = v8['Low'].astype(float)
v8_h = v8['High'].astype(float)

In [23]:
# -- --------------------------------------Ventana indicador 9 ---------------------------------- #

v9 = ventana(F1 = "2018-08-10T11:40:00Z",F2 = "2018-08-10T13:30:00Z")
v9_o = v9['Open'].astype(float)
v9_c = v9['Close'].astype(float)
v9_l = v9['Low'].astype(float)
v9_h = v9['High'].astype(float)

In [24]:
# -- --------------------------------------Ventana indicador 10 ---------------------------------- #

v10 = ventana(F1 = "2018-07-12T11:40:00Z",F2 = "2018-07-12T13:20:00Z")
v10_o = v10['Open'].astype(float)
v10_c = v10['Close'].astype(float)
v10_l = v10['Low'].astype(float)
v10_h = v10['High'].astype(float)

In [25]:
# -- --------------------------------------Ventana indicador 11 ---------------------------------- #

v11 = ventana(F1 = "2018-06-12T11:40:00Z",F2 = "2018-06-12T13:30:00Z")
v11_o = v11['Open'].astype(float)
v11_c = v11['Close'].astype(float)
v11_l = v11['Low'].astype(float)
v11_h = v11['High'].astype(float)

In [26]:
# -- --------------------------------------Ventana indicador 12 ---------------------------------- #

v12 = ventana(F1 = "2018-05-10T11:40:00Z",F2 = "2018-05-10T13:30:00Z")
v12_o = v12['Open'].astype(float)
v12_c = v12['Close'].astype(float)
v12_l = v12['Low'].astype(float)
v12_h = v12['High'].astype(float)

In [27]:
# -- --------------------------------------Ventana indicador 13 ---------------------------------- #

v13 = ventana(F1 = "2018-04-11T11:40:00Z",F2 = "2018-04-11T13:30:00Z")
v13_o = v13['Open'].astype(float)
v13_c = v13['Close'].astype(float)
v13_l = v13['Low'].astype(float)
v13_h = v13['High'].astype(float)

In [28]:
# -- --------------------------------------Ventana indicador 14 ---------------------------------- #

v14 = ventana(F1 = "2018-03-13T11:40:00Z",F2 = "2018-03-13T13:30:00Z")
v14_o = v14['Open'].astype(float)
v14_c = v14['Close'].astype(float)
v14_l = v14['Low'].astype(float)
v14_h = v14['High'].astype(float)

In [29]:
# -- --------------------------------------Ventana indicador 15 ---------------------------------- #

v15 = ventana(F1 = "2018-02-14T11:40:00Z",F2 = "2018-02-14T13:30:00Z")
v15_o = v15['Open'].astype(float)
v15_c = v15['Close'].astype(float)
v15_l = v15['Low'].astype(float)
v15_h = v15['High'].astype(float)

In [30]:
# -- --------------------------------------Ventana indicador 16 ---------------------------------- #

v16 = ventana(F1 = "2018-01-10T12:40:00Z",F2 = "2018-01-10T14:30:00Z")
v16_o = v16['Open'].astype(float)
v16_c = v16['Close'].astype(float)
v16_l = v16['Low'].astype(float)
v16_h = v16['High'].astype(float)

In [31]:
# -- --------------------------------------Ventana indicador 17 ---------------------------------- #

v17 = ventana(F1 = "2017-12-13T12:40:00Z",F2 = "2017-12-13T14:30:00Z")
v17_o = v17['Open'].astype(float)
v17_c = v17['Close'].astype(float)
v17_l = v17['Low'].astype(float)
v17_h = v17['High'].astype(float)

In [32]:
# -- --------------------------------------Ventana indicador 18 ---------------------------------- #

v18 = ventana(F1 = "2017-11-15T12:40:00Z",F2 = "2017-11-15T14:30:00Z")
v18_o = v18['Open'].astype(float)
v18_c = v18['Close'].astype(float)
v18_l = v18['Low'].astype(float)
v18_h = v18['High'].astype(float)

In [33]:
# -- --------------------------------------Ventana indicador 19 ---------------------------------- #

v19 = ventana(F1 = "2017-10-13T11:40:00Z",F2 = "2017-10-13T13:30:00Z")
v19_o = v19['Open'].astype(float)
v19_c = v19['Close'].astype(float)
v19_l = v19['Low'].astype(float)
v19_h = v19['High'].astype(float)

In [34]:
# -- --------------------------------------Ventana indicador 20 ---------------------------------- #

v20 = ventana(F1 = "2017-09-14T11:40:00Z",F2 = "2017-09-14T13:30:00Z")
v20_o = v20['Open'].astype(float)
v20_c = v20['Close'].astype(float)
v20_l = v20['Low'].astype(float)
v20_h = v20['High'].astype(float)

In [35]:
# -- --------------------------------------Ventana indicador 21 ---------------------------------- #

v21 = ventana(F1 = "2017-09-14T11:40:00Z",F2 = "2017-09-14T13:30:00Z")
v21_o = v21['Open'].astype(float)
v21_c = v21['Close'].astype(float)
v21_l = v21['Low'].astype(float)
v21_h = v21['High'].astype(float)

In [36]:
# -- --------------------------------------Ventana indicador 22 ---------------------------------- #

v22 = ventana(F1 = "2017-07-14T11:40:00Z",F2 = "2017-07-14T13:30:00Z")
v22_o = v22['Open'].astype(float)
v22_c = v22['Close'].astype(float)
v22_l = v22['Low'].astype(float)
v22_h = v22['High'].astype(float)

In [37]:
# -- --------------------------------------Ventana indicador 23 ---------------------------------- #

v23 = ventana(F1 = "2017-06-14T11:40:00Z",F2 = "2017-06-14T13:30:00Z")
v23_o = v23['Open'].astype(float)
v23_c = v23['Close'].astype(float)
v23_l = v23['Low'].astype(float)
v23_h = v23['High'].astype(float)

In [38]:
# -- --------------------------------------Ventana indicador 25 ---------------------------------- #

v25 = ventana(F1 = "2017-04-14T11:40:00Z",F2 = "2017-04-14T13:30:00Z")
v25_o = v25['Open'].astype(float)
v25_c = v25['Close'].astype(float)
v25_l = v25['Low'].astype(float)
v25_h = v25['High'].astype(float)

___
### <Strong> <font color = skyblue> DataFrame5 Escenario-Caracteristicas </Strong>

In [73]:
indice = tabla['Scene']
columnas = ['A','B', 'C', 'D']
df = pd.DataFrame(index=indice, columns=columnas)
df = df.fillna(0) # -- llenar todas las celdas con 0s
df

,A,B,C,D
Scene,,,,
C,0,0,0,0
C,0,0,0,0
A,0,0,0,0
A,0,0,0,0
A,0,0,0,0
A,0,0,0,0
C,0,0,0,0
C,0,0,0,0
A,0,0,0,0
